In [1]:
#general imports
import sys
sys.path.append(r'C:\Users\Robert Lees\Documents\Code\Vape\suite2p_etc')
sys.path.append('..')

import numpy as np
import os
import time

In [2]:
#notebook specific imports
from utils.gsheets_importer import gsheet2df, correct_behaviour_df, split_df, path_conversion, path_finder
from utils.artifact_removal import artifact_removal
from utils.utils_funcs import *
from utils.paq2py import *
from utils.parse_metadata import *
import tifffile
import copy
import math

import suite2p
print(suite2p.__path__)
from suite2p.run_s2p import run_s2p
from settings import ops

import matplotlib.pyplot as plt

['C:\\ProgramData\\Anaconda3\\lib\\site-packages\\suite2p']


In [8]:
# read g sheets to populate some lists for processing stim artifact and running suite2p

sheet_ID = '1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY'
SHEET_NAME = '2019-07-15_RL052'
df = gsheet2df(sheet_ID, HEADER_ROW=2, SHEET_NAME=SHEET_NAME)

# at this point we have lots of files that could be whisker stim or photostim, need to find out which is which

for_processing = split_df(df, 's2p_me') # only files with TRUE in suite2p_me column

if not for_processing.shape[0]:
    raise Exception('ERROR: no files set for processing')

stim = for_processing.loc[:,'stim'] # find out what stims have been carried out
photostim_idx = [i for i,stim in enumerate(stim) if stim=='p'] # row indices of all photostim exps (for artifact removal)
whisker_stim_idx = [i for i,stim in enumerate(stim) if stim=='w'] # '' for whisker stim (no artifact removal)

if ( len(photostim_idx) + len(whisker_stim_idx) ) != stim.shape[0]:
    raise Exception('ERROR: stim type is not defined for some files')

tiff_paths = for_processing.loc[:,'tiff_path']
naparm_paths = for_processing.loc[:,'naparm_path']

# if not all(tiff_paths) or not all(naparm_paths):
#     raise Exception('ERROR: missing tiff or naparm paths')
    
packerstation_path = r"P:" # the path to PackerStation on the local machine
# TODO: update this to path_finder rather than conversion, to increase failsafe at this point
# SOMETIMES THERE ARE NO NAPARMS
tiffs_pstation = path_conversion(tiff_paths, packerstation_path) # convert paths (from Packer1 or PackerStation) to local PackerStation paths
naparm_pstation = path_conversion(naparm_paths, packerstation_path)

print(tiffs_pstation, naparm_pstation)

['P:rlees\\Data\\2019-07-15\\2019-07-15_RL052\\2019-07-15_RL052_t-007', 'P:rlees\\Data\\2019-07-15\\2019-07-15_RL052\\2019-07-15_RL052_t-008'] ['P:rlees\\Data\\2019-07-15\\2019-07-15_RL052\\NAPARM\\2019-07-15_RL052_NAPARM_003', 'P:rlees\\Data\\2019-07-15\\2019-07-15_RL052\\NAPARM\\2019-07-15_RL052_NAPARM_004']


In [4]:
pv_values, naparm_xml, naparm_gpl = getMetadata(tiffs_pstation, naparm_pstation)

P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-007
P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-007\2019-07-15_RL052_t-007_BACKUP.xml
P:rlees\Data\2019-07-15\2019-07-15_RL052\NAPARM\2019-07-15_RL052_NAPARM_003\2019-07-15_RL052_NAPARM_003.xml
P:rlees\Data\2019-07-15\2019-07-15_RL052\NAPARM\2019-07-15_RL052_NAPARM_003\2019-07-15_RL052_NAPARM_003.gpl
Number of frames: 37650 
Number of planes: 1
Frames per second: 24.28985806610077
Frame averaging: 1
Size (x): 974
Size (y): 632
Imaging laser power: 135.897435897436
Pixel size (x,y): 1.31338993281181 1.31338993281181
Number of groups: 4 
Number of sequence reps: 10 
Number of shots: 1 
Numbers of trials: 100 
Inter-point delay: 5
Repetitions: 1
Single stim dur (ms): 10
P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-008
P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-008\2019-07-15_RL052_t-008_BACKUP.xml
P:rlees\Data\2019-07-15\2019-07-15_RL052\NAPARM\2019-07-15_RL052_NAPARM_004\2019-07-15

In [5]:
# obtain list of tiffs to run stim removal on

tiff_lists = []

# need to find single multi-page TIFF or many TIFFs/MPTIFFs

for tiff in tiffs_pstation:
    print(tiff)
    
    if '.tif' not in tiff or '.tiff' not in tiff: # if the path is not directly a TIFF, it may be folder with MPTIFF or multiple TIFFs/MPTIFFs
        
        items = os.listdir(tiff)
        newlist = []
        
        for name in items:
            if name.endswith(".tiff") or name.endswith(".tif"):
                filename = os.path.join(tiff, name)
                newlist.append(filename)
        tiff_lists.append(newlist)
        
    else: # if provided path directs to TIFF file, make sure it is only one in folder
        
        tiff_count = 0
        
        parent_dir = os.path.dirname(tiff)
        items = os.listdir(parent_dir)
                
        for name in items:
            if name.endswith(".tiff") or name.endswith(".tif"):
                tiff_count += 1
        
        if tiff_count > 1:
            raise Exception('ERROR: make sure large, single TIFF files are in their own folder')
        else: 
            tiff_lists.append(tiff)
            
photostim_tiffs = [tiff_lists[i] for i in photostim_idx] # only run artifact removal-specific code on photostim exps

P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-007
P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-008


In [6]:
# important: data paths must be lists even if only one element
# can also only run on specified tiffs

user_batch_size = 200 # number of frames to be processed at once (i.e. registered)

db = []

for i,tiff_list in enumerate(tiff_lists):
    print(tiff_list[0])
    
    if tiff_list in photostim_tiffs: # photostim experiments should have tiffs processed with artifact removed
        print('Photostim experiment')
        umbrella_folder = os.path.dirname(tiff_list[0])
#         folder_name = path_finder(umbrella_folder, 'Artifact_removed',  is_folder=True) # find new folder containing artifact-removed data
        folder_name = umbrella_folder
    else: # whisker stim experiments
        print('Whisker stim experiment')
        umbrella_folder = os.path.dirname(tiff_list[0])
#         folder_name = path_finder(umbrella_folder, 'Artifact_removed',  is_folder=True) # find new folder containing artifact-removed data
        folder_name = umbrella_folder
        
#     tiff_list = []
    
#     for file in os.listdir(folder_name):
#             if '.ome' not in file and '.tif' in file:
#                 tiff_list.append(file) 
    
#     tiff_list = sorted(tiff_list)
    
    fps = pv_values[i][0]
    n_planes = pv_values[i][5]
    pixel_x = pv_values[i][7]
    pixel_y = pv_values[i][8]
    frame_size = pv_values[i][2:4]
    num_pixels = frame_size[0]*frame_size[1]
    
    sampling_rate = fps/n_planes
    diameter_x = 13/pixel_x
    diameter_y = 13/pixel_y
    diameter = int(diameter_x), int(diameter_y)
    batch_size = user_batch_size * (262144 / num_pixels) # larger frames will be more RAM intensive, scale user batch size based on num pixels in 512x512 images
      
    db.append({ 'data_path' : [folder_name], 
#               'tiff_list' : tiff_list,
              'fs' : float(sampling_rate),
              'diameter' : diameter, 
              'batch_size' : int(batch_size), 
              'nimg_init' : int(batch_size),
              'nplanes' : n_planes
              })
    
db

P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-007\2019-07-15_RL052_t-007_Cycle00001_Ch3.tif
Photostim experiment
P:rlees\Data\2019-07-15\2019-07-15_RL052\2019-07-15_RL052_t-008\2019-07-15_RL052_t-008_Cycle00001_Ch3.tif
Photostim experiment


[{'data_path': ['P:rlees\\Data\\2019-07-15\\2019-07-15_RL052\\2019-07-15_RL052_t-007'],
  'fs': 24.28985806610077,
  'diameter': (9, 9),
  'batch_size': 85,
  'nimg_init': 85,
  'nplanes': 1},
 {'data_path': ['P:rlees\\Data\\2019-07-15\\2019-07-15_RL052\\2019-07-15_RL052_t-008'],
  'fs': 24.28985806610077,
  'diameter': (9, 9),
  'batch_size': 85,
  'nimg_init': 85,
  'nplanes': 1}]

In [7]:
t1 = time.time()

for dbi in db:
    opsEnd = run_s2p(ops=ops,db=dbi)
    
t2 = time.time()
t2 - t1

Found 1 tifs


C:\ProgramData\Anaconda3\lib\site-packages\skimage\external\tifffile\tifffile.py:1635: UserWarning: can not reshape (2000, 632, 974) to (37650, 632, 974)
  warnings.warn(str(e))


37650
time 784.8175. Wrote tifs to binaries for 1 planes
C:\BIN\suite2p\plane0\data_raw.bin
computed reference frame for registration
registered 425/37650 frames in time 111.40
registered 850/37650 frames in time 223.72
registered 1275/37650 frames in time 336.56
registered 1700/37650 frames in time 449.21
registered 2125/37650 frames in time 562.16
registered 2550/37650 frames in time 676.45
registered 2975/37650 frames in time 787.77
registered 3400/37650 frames in time 897.55
registered 3825/37650 frames in time 1006.34
registered 4250/37650 frames in time 1117.55
registered 4675/37650 frames in time 1228.23
registered 5100/37650 frames in time 1338.65
registered 5525/37650 frames in time 1451.78
registered 5950/37650 frames in time 1560.06
registered 6375/37650 frames in time 1668.76
registered 6800/37650 frames in time 1778.62
registered 7225/37650 frames in time 1890.33
registered 7650/37650 frames in time 2002.21
registered 8075/37650 frames in time 2113.35
registered 8500/37650

C:\ProgramData\Anaconda3\lib\site-packages\skimage\external\tifffile\tifffile.py:1635: UserWarning: can not reshape (2000, 632, 974) to (37650, 632, 974)
  warnings.warn(str(e))


37650
time 707.6765. Wrote tifs to binaries for 1 planes
C:\BIN\suite2p\plane0\data_raw.bin
computed reference frame for registration
registered 425/37650 frames in time 108.43
registered 850/37650 frames in time 216.32
registered 1275/37650 frames in time 324.15
registered 1700/37650 frames in time 431.64
registered 2125/37650 frames in time 539.21
registered 2550/37650 frames in time 646.07
registered 2975/37650 frames in time 753.72
registered 3400/37650 frames in time 861.74
registered 3825/37650 frames in time 969.04
registered 4250/37650 frames in time 1076.70
registered 4675/37650 frames in time 1184.20
registered 5100/37650 frames in time 1291.92
registered 5525/37650 frames in time 1399.49
registered 5950/37650 frames in time 1506.85
registered 6375/37650 frames in time 1614.46
registered 6800/37650 frames in time 1721.82
registered 7225/37650 frames in time 1829.39
registered 7650/37650 frames in time 1937.12
registered 8075/37650 frames in time 2044.45
registered 8500/37650 

24171.488065958023